In [ ]:
# spk_students.py

import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# =========================
# 1. KONFIGURASI UMUM
# =========================

DATA_PATH = "Students_Performance_data_set.xlsx"

# 20 kriteria yang dipakai (harus sama persis dengan nama kolom di file)
CRITERIA = [
    "AdmYear",
    "Age",
    "HSCYear",
    "Semester",
    "Scholarship",
    "Study Hours",
    "Study Frequency",
    "Learn Mode",
    "Social Media",
    "English Skill",
    "Attendance",
    "Probation",
    "Consultancy",
    "Skill Hours",
    "Co Curricular",
    "Health Issues",
    "SGPA",
    "CGPA",
    "Credits",
    "Family Income"
]

# -------------------------
# Tipe kriteria untuk TOPSIS
# "benefit"  : semakin besar semakin baik
# "cost"     : semakin besar semakin buruk
# >>> SESUAIKAN dengan tabel atribut yang sudah kamu buat di Excel <<<
# -------------------------
CRITERIA_TYPES = [
    "benefit",   # AdmYear
    "cost",      # Age  (misalnya: makin tua makin berisiko -> sesuaikan kalau perlu)
    "benefit",   # HSCYear (lebih baru -> lebih baik)
    "cost",   # Semester
    "cost",   # Scholarship (punya beasiswa -> lebih baik)
    "cost",   # Study Hours
    "cost",   # Study Frequency
    "cost",   # Learn Mode (misal: mode tertentu lebih baik, tergantung encoding)
    "benefit",      # Social Media (banyak waktu di sosmed -> buruk)
    "cost",   # English Skill
    "cost",   # Attendance
    "benefit",      # Probation (pernah probation -> buruk)
    "cost",   # Consultancy (pernah konsultasi dosen -> baik)
    "cost",   # Skill Hours
    "cost",   # Co Curricular
    "benefit",      # Health Issues (punya masalah kesehatan -> buruk)
    "cost",   # SGPA
    "cost",   # CGPA
    "cost",   # Credits
    "cost"    # Family Income (anggap lebih tinggi -> mendukung, sesuaikan kalau perlu)
]

# -------------------------
# Bobot AHP
# Harus berisi 20 nilai, urutan HARUS sama dengan CRITERIA di atas.
# Untuk sementara, isi dummy dulu (rata), NANTI GANTI dengan hasil AHP kamu.
# -------------------------
ahp_weights = np.array([
    0.02,  # AdmYear
    0.02,  # Age
    0.02,  # HSCYear
    0.02,  # Semester
    0.03,  # Scholarship
    0.05,  # Study Hours
    0.05,  # Study Frequency
    0.03,  # Learn Mode
    0.04,  # Social Media
    0.04,  # English Skill
    0.06,  # Attendance
    0.05,  # Probation
    0.04,  # Consultancy
    0.04,  # Skill Hours
    0.03,  # Co Curricular
    0.04,  # Health Issues
    0.07,  # SGPA
    0.07,  # CGPA
    0.04,  # Credits
    0.04   # Family Income
], dtype=float)

# normalisasi bobot (jaga-jaga kalau belum pas 1)
ahp_weights = ahp_weights / ahp_weights.sum()

# =========================
# 2. FUNGSI PREPROCESSING
# =========================

def load_and_select_features(path: str, criteria: list) -> pd.DataFrame:
    df = pd.read_excel(path)
    # ambil hanya 20 fitur yang dipakai
    df = df[criteria].copy()
    return df

def encode_categorical(df: pd.DataFrame) -> pd.DataFrame:
    """
    Encoding sederhana untuk fitur kategorikal.
    SESUAIKAN mapping ini dengan isi dataset kamu.
    """
    df = df.copy()

    # Scholarship: Yes/No -> 1/0
    if "Scholarship" in df.columns:
        df["Scholarship"] = df["Scholarship"].map({"Yes": 1, "No": 0}).fillna(0)

    # Learn Mode: Offline/Online (misal) -> 1/0
    if "Learn Mode" in df.columns:
        df["Learn Mode"] = df["Learn Mode"].map({"Offline": 1, "Online": 0}).fillna(0)

    # Probation: Yes/No -> 1/0
    if "Probation" in df.columns:
        df["Probation"] = df["Probation"].map({"Yes": 1, "No": 0}).fillna(0)

    # Consultancy: Yes/No -> 1/0
    if "Consultancy" in df.columns:
        df["Consultancy"] = df["Consultancy"].map({"Yes": 1, "No": 0}).fillna(0)

    # Co Curricular: Yes/No -> 1/0
    if "Co Curricular" in df.columns:
        df["Co Curricular"] = df["Co Curricular"].map({"Yes": 1, "No": 0}).fillna(0)

    # Health Issues: Yes/No -> 1/0
    if "Health Issues" in df.columns:
        df["Health Issues"] = df["Health Issues"].map({"Yes": 1, "No": 0}).fillna(0)

    # English Skill: Basic / Intermediate / Advance -> 1 / 2 / 3
    if "English Skill" in df.columns:
        df["English Skill"] = df["English Skill"].map({
            "Basic": 1,
            "Intermediate": 2,
            "Advance": 3
        }).fillna(2)  # default Intermediate

    # Pastikan numeric
    for col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

    # Drop baris yang masih ada NaN setelah encoding
    df = df.dropna().reset_index(drop=True)
    return df

def scale_features(df: pd.DataFrame) -> (pd.DataFrame, MinMaxScaler):
    scaler = MinMaxScaler()
    scaled = scaler.fit_transform(df.values)
    df_scaled = pd.DataFrame(scaled, columns=df.columns)
    return df_scaled, scaler

# =========================
# 3. K-MEANS CLUSTERING
# =========================

def run_kmeans(df_scaled: pd.DataFrame, n_clusters: int = 3, random_state: int = 42):
    kmeans = KMeans(n_clusters=n_clusters, n_init=10, random_state=random_state)
    labels = kmeans.fit_predict(df_scaled.values)
    sil = silhouette_score(df_scaled.values, labels)
    return kmeans, labels, sil

def find_high_risk_cluster(df_original: pd.DataFrame, labels: np.ndarray):
    """
    Anggap cluster dengan rata-rata CGPA TERENDAH = cluster risiko tinggi.
    Kalau kamu mau pakai indikator lain (SGPA + Attendance), silakan ganti logic ini.
    """
    df = df_original.copy()
    df["cluster"] = labels

    if "CGPA" not in df.columns:
        raise ValueError("Kolom 'CGPA' tidak ditemukan di data.")

    cluster_means = df.groupby("cluster")["CGPA"].mean()
    high_risk_cluster = cluster_means.idxmin()
    return high_risk_cluster, df

# =========================
# 4. TOPSIS
# =========================

def topsis(decision_matrix: np.ndarray,
           weights: np.ndarray,
           criteria_types: list):
    """
    decision_matrix : numpy (m x n)  -> m alternatif, n kriteria
    weights         : numpy (n,)     -> bobot AHP
    criteria_types  : list of 'benefit' / 'cost' length n
    """
    m, n = decision_matrix.shape

    # Normalisasi (vector normalization)
    norm_matrix = decision_matrix / np.sqrt((decision_matrix ** 2).sum(axis=0))

    # Bobot
    weighted_matrix = norm_matrix * weights

    # Solusi ideal + dan -
    ideal_pos = np.zeros(n)
    ideal_neg = np.zeros(n)

    for j in range(n):
        if criteria_types[j] == "benefit":
            ideal_pos[j] = weighted_matrix[:, j].max()
            ideal_neg[j] = weighted_matrix[:, j].min()
        else:  # cost
            ideal_pos[j] = weighted_matrix[:, j].min()
            ideal_neg[j] = weighted_matrix[:, j].max()

    # Jarak ke solusi ideal
    dist_pos = np.sqrt(((weighted_matrix - ideal_pos) ** 2).sum(axis=1))
    dist_neg = np.sqrt(((weighted_matrix - ideal_neg) ** 2).sum(axis=1))

    # Nilai preferensi Vi
    vi = dist_neg / (dist_pos + dist_neg)

    return vi, dist_pos, dist_neg

# =========================
# 5. MAIN PIPELINE
# =========================

def main():
    # 1) Load & pilih 20 fitur
    df_raw = load_and_select_features(DATA_PATH, CRITERIA)
    print(f"Data awal: {df_raw.shape[0]} baris, {df_raw.shape[1]} kolom")

    # 2) Encoding kategori -> numerik
    df_encoded = encode_categorical(df_raw)
    print(f"Setelah encoding & drop NaN: {df_encoded.shape[0]} baris")

    # 3) Scaling (Min-Max)
    df_scaled, scaler = scale_features(df_encoded)

    # 4) K-Means
    kmeans, labels, sil = run_kmeans(df_scaled, n_clusters=3)
    print(f"Silhouette Score clustering: {sil:.4f}")

    # 5) Tentukan cluster risiko tinggi (CGPA terendah)
    high_risk_cluster, df_with_cluster = find_high_risk_cluster(df_encoded, labels)
    print(f"Cluster risiko tinggi = {high_risk_cluster}")

    # Ambil hanya mahasiswa di cluster risiko tinggi
    df_risk = df_with_cluster[df_with_cluster["cluster"] == high_risk_cluster].copy()
    df_risk = df_risk.drop(columns=["cluster"])

    print(f"Jumlah mahasiswa di cluster risiko tinggi: {df_risk.shape[0]}")

    # 6) Siapkan matriks keputusan untuk TOPSIS
    #    (pakai data yang sudah discale tapi subset baris cluster risiko tinggi)
    #    -> supaya sesuai kriteria 1:1 dengan bobot AHP
    df_scaled_with_cluster = df_scaled.copy()
    df_scaled_with_cluster["cluster"] = labels
    dm_scaled_risk = df_scaled_with_cluster[
        df_scaled_with_cluster["cluster"] == high_risk_cluster
    ].drop(columns=["cluster"]).values

    if dm_scaled_risk.shape[1] != len(ahp_weights):
        raise ValueError(
            f"Jumlah kolom keputusan ({dm_scaled_risk.shape[1]}) "
            f"tidak sama dengan jumlah bobot AHP ({len(ahp_weights)})"
        )

    # 7) TOPSIS
    vi, d_pos, d_neg = topsis(dm_scaled_risk, ahp_weights, CRITERIA_TYPES)

    # Tambahkan hasil ke dataframe df_risk
    df_risk = df_risk.reset_index(drop=True)
    df_risk["D_plus"] = d_pos
    df_risk["D_minus"] = d_neg
    df_risk["Vi"] = vi

    # 8) Ranking (semakin besar Vi -> prioritas bimbingan lebih tinggi)
    df_risk["Rank"] = df_risk["Vi"].rank(ascending=False, method="dense").astype(int)
    df_risk_sorted = df_risk.sort_values(by="Vi", ascending=False)

    # 9) Tampilkan 10 teratas
    print("\nTop 10 mahasiswa yang diprioritaskan untuk bimbingan tambahan:")
    cols_show = ["AdmYear", "Age", "Semester", "SGPA", "CGPA", "Attendance", "Vi", "Rank"]
    cols_show = [c for c in cols_show if c in df_risk_sorted.columns]
    print(df_risk_sorted[cols_show].head(10))

    # Kalau mau simpan ke Excel:
    df_risk_sorted.to_excel("hasil_prioritas_bimbingan.xlsx", index=False)
    print("\nFile hasil_prioritas_bimbingan.xlsx telah disimpan.")

if __name__ == "__main__":
    main()